# Crawling Data PTA dan Berita

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re, sys, time

In [ ]:
BASE_URL = "https://pta.trunojoyo.ac.id/c_search/byprod"

In [2]:
# Daftar prodi Fakultas Teknik
PRODI_FATEK = {
    10: "Teknik Informatika",
    11: "Teknik Industri",
    12: "Teknik Mesin",
    13: "Teknik Elektro",
    14: "Teknik Mekatronika",
    15: "Sistem Informasi",
    16: "Teknik Multimedia dan Jaringan",
    17: "Manajemen Informatika",
    18: "Mekatronika"
}

In [3]:
# Batas maksimal halaman per prodi (biar tidak lama)
MAX_PAGE_LIMIT = 10   # <-- ganti jadi 10 kalau mau lebih banyak

In [9]:
def get_max_page(prodi_id, BASE_URL):
    url = f"{BASE_URL}/{prodi_id}/1"
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")

    last_page = soup.select_one('ol.pagination a:contains("»")')
    if last_page and "href" in last_page.attrs:
        href = last_page["href"]
        max_page = int(href.split("/")[-1])
        return max_page
    return 1

In [5]:
def print_progress(prodi_id, prodi, current_page, total_pages):
    percent = (current_page / total_pages) * 100
    bar_length = 20
    filled_length = int(bar_length * current_page // total_pages)
    bar = '█' * filled_length + '-' * (bar_length - filled_length)
    sys.stdout.write(f'\r[{prodi_id}] {prodi} - Page {current_page}/{total_pages} [{bar}] {percent:.2f}%')
    sys.stdout.flush()
    if current_page == total_pages:
        sys.stdout.write('\n')

In [10]:
def pta_fatek_fast(BASE_URL):
    start_time = time.time()

    data = {
        "id": [], "penulis": [], "judul": [], "abstrak_id": [],
        "abstrak_en": [], "pembimbing_pertama": [], "pembimbing_kedua": [],
        "prodi": []
    }

    for prodi_id, prodi_name in PRODI_FATEK.items():
        max_page = min(get_max_page(prodi_id, BASE_URL), MAX_PAGE_LIMIT)  # batasi halaman
        for j in range(1, max_page + 1):
            url = f"{BASE_URL}/{prodi_id}/{j}"
            r = requests.get(url)
            soup = BeautifulSoup(r.content, "html.parser")
            jurnals = soup.select('li[data-cat="#luxury"]')

            for jurnal in jurnals:
                link_keluar = jurnal.select_one('a.gray.button')['href']
                id_match = re.search(r"/detail/(\d+)", link_keluar)
                pta_id = id_match.group(1) if id_match else None

                response = requests.get(link_keluar)
                soup1 = BeautifulSoup(response.content, "html.parser")
                isi = soup1.select_one('div#content_journal')

                judul = isi.select_one('a.title').text.strip() if isi.select_one('a.title') else "N/A"
                penulis = isi.find(string=re.compile("Penulis")) if isi else "N/A"
                pembimbing_pertama = isi.find(string=re.compile("Dosen Pembimbing I")) if isi else "N/A"
                pembimbing_kedua = isi.find(string=re.compile("Dosen Pembimbing II")) if isi else "N/A"

                paragraf = isi.select('p[align="justify"]') if isi else []
                abstrak_id = paragraf[0].get_text(strip=True) if len(paragraf) > 0 else "N/A"
                abstrak_en = paragraf[1].get_text(strip=True) if len(paragraf) > 1 else "N/A"

                data["id"].append(pta_id)
                data["penulis"].append(penulis)
                data["judul"].append(judul)
                data["abstrak_id"].append(abstrak_id)
                data["abstrak_en"].append(abstrak_en)
                data["pembimbing_pertama"].append(pembimbing_pertama)
                data["pembimbing_kedua"].append(pembimbing_kedua)
                data["prodi"].append(prodi_name)

            print_progress(prodi_id, prodi_name, j, max_page)

        sys.stdout.write("\n")

    # simpan hasil
    df = pd.DataFrame(data)
    df.to_csv("pta_fatek_fast.csv", index=False, encoding="utf-8-sig")

    elapsed = int(time.time() - start_time)
    jam, sisa = divmod(elapsed, 3600)
    menit, detik = divmod(sisa, 60)

    print("\n✅ Data Fakultas Teknik berhasil dikumpulkan!")
    print(f"📊 Total entri: {len(df)}")
    print(f"⏱️ Waktu eksekusi: {jam} jam {menit} menit {detik} detik")

    return df

In [13]:
pta_fatek_fast(BASE_URL)

/usr/local/lib/python3.12/dist-packages/soupsieve/css_parser.py:876: FutureWarning: The pseudo class ':contains' is deprecated, ':-soup-contains' should be used moving forward.
  warnings.warn(  # noqa: B028


[10] Teknik Informatika - Page 10/10 [████████████████████] 100.00%

[11] Teknik Industri - Page 10/10 [████████████████████] 100.00%

[12] Teknik Mesin - Page 10/10 [████████████████████] 100.00%

[13] Teknik Elektro - Page 10/10 [████████████████████] 100.00%

[14] Teknik Mekatronika - Page 10/10 [████████████████████] 100.00%

[15] Sistem Informasi - Page 10/10 [████████████████████] 100.00%

[16] Teknik Multimedia dan Jaringan - Page 10/10 [████████████████████] 100.00%

[17] Manajemen Informatika - Page 10/10 [████████████████████] 100.00%

[18] Mekatronika - Page 10/10 [████████████████████] 100.00%


✅ Data Fakultas Teknik berhasil dikumpulkan!
📊 Total entri: 450
⏱️ Waktu eksekusi: 0 jam 53 menit 58 detik


,id,penulis,judul,abstrak_id,abstrak_en,pembimbing_pertama,pembimbing_kedua,prodi
0,040411100468,Penulis : A.Ubaidillah S.Kom,PERANCANGAN DAN IMPLEMENTASI SISTEM DATABASE \...,Sistem informasi akademik (SIAKAD) merupaka...,Academic information systems (SIAKAD) is an in...,Dosen Pembimbing I : Budi Setyono M.T,Dosen Pembimbing II :Hermawan S.T,Teknik Informatika
1,040411100476,"Penulis : M. Basith Ardianto,",APLIKASI KONTROL DAN MONITORING JARINGAN KOMPU...,Berjalannya koneksi jaringan komputer dengan l...,-,"Dosen Pembimbing I : Drs. Budi Soesilo, MT","Dosen Pembimbing II :Koko Joni, ST",Teknik Informatika
2,040411100480,"Penulis : Akhmad Suyandi, S.Kom",RANCANG BANGUN APLIKASI PROXY SERVER UNTUK\r\n...,Web server adalah sebuah perangkat lunak serve...,Web server is a server software functioning to...,"Dosen Pembimbing I : Drs. Budi Soesilo, M.T","Dosen Pembimbing II :Hermawan, ST, MT",Teknik Informatika
3,070411100070,Penulis : Heri Supriyanto,SISTEM PENDUKUNG KEPUTUSAN OPTIMASI PENJADWALA...,Penjadwalan kuliah di Perguruan Tinggi me...,Scheduling courses in universities is a ...,"Dosen Pembimbing I : Mulaab, S.Si., M.Kom","Dosen Pembimbing II :Firli Irhamni, ST., M.Kom",Teknik Informatika
4,080411100115,Penulis : Septian Rahman Hakim,SISTEM AUGMENTED REALITY ANIMASI BENDA BERGERA...,Seiring perkembangan teknologi yang ada diduni...,As the development of technology existing in t...,"Dosen Pembimbing I : Arik Kurniawati, S.Kom., ...","Dosen Pembimbing II :Haryanto, S.T., M.T.",Teknik Informatika
...,...,...,...,...,...,...,...,...
445,110611100051,Penulis : windi putri friska,Problematika Ket,Penelitian ini bertujuan untuk mengetahui peny...,The purpose of the study is to determnine the ...,"Dosen Pembimbing I : Hani'ah, S.Pd., M.Pd","Dosen Pembimbing II :Wanda Ramansyah, S.Pd., M.Pd",Mekatronika
446,110611100073,Penulis : RIRIN HIDAYATI,PENERAPAN STRATEGI PEMBELAJARAN KONTEKSTUAL BE...,Ririn Hidayati. 2015. Penerapan Strategi Pembe...,Ririn Hidayati. 2015. The Application of Conte...,"Dosen Pembimbing I : PRIYONO TRI FEBRIANTO, S....","Dosen Pembimbing II :MUHAMMAD BUSYRO KARIM, S....",Mekatronika
447,110611100027,Penulis : SEFTY WAHYU MULYANINGSIH,Perbandingan Hasil Belajar Dan Efektivitas Pem...,ABSTRAK\r\nPenelitian ini bertujuan untuk meng...,ABSTRACT\r\nThis study aims to determine the d...,"Dosen Pembimbing I : Ariesta Kartika Sari, S.S...","Dosen Pembimbing II :Moh. Edy Nurtamam, S.Pd.,...",Mekatronika
448,110611100077,Penulis : Dwi Juwita Sari,PENGARUH MODEL PEMBELAJARAN KOOPERATIF \r\nTIP...,Dwi Juwita Sari. 2015. Pengaruh Model Pembelaj...,Dwi Juwita Sari. 2015. The Effect of Cooperati...,"Dosen Pembimbing I : Eva Ari Wahyuni, S.Pd., M.Si","Dosen Pembimbing II :Ira Fatmawati, S.S., M.Pd",Mekatronika


## Page & Link Keluar PTA

In [14]:
# Progress bar
def print_progress(prodi_id, prodi, current_page, total_pages):
    percent = (current_page / total_pages) * 100
    bar_length = 20
    filled_length = int(bar_length * current_page // total_pages)
    bar = '█' * filled_length + '-' * (bar_length - filled_length)
    sys.stdout.write(f'\r[{prodi_id}] {prodi} - Page {current_page}/{total_pages} [{bar}] {percent:.2f}%')
    sys.stdout.flush()
    if current_page == total_pages:
        sys.stdout.write('\n\n')

# Crawling link page + link keluar
def pta_links():
    start_time = time.time()  # mulai hitung waktu

    data = {
        "no": [],
        "page": [],
        "link_keluar": []
    }

    no = 1  # nomor urut

    for i in range(1, 42):  # looping semua prodi (id 1–41)
        total_pages = 3  # bisa diubah sesuai kebutuhan (misal 10 halaman / semua halaman)
        prodi_name = None

        for j in range(1, total_pages + 1):  # loop halaman per prodi
            url = f"https://pta.trunojoyo.ac.id/c_search/byprod/{i}/{j}"
            r = requests.get(url)
            soup = BeautifulSoup(r.content, "html.parser")
            jurnals = soup.select('li[data-cat="#luxury"]')

            isii = soup.select_one('div#begin')
            if not isii:
                continue
            prodi_full = isii.select_one('h2').text.strip()
            prodi = prodi_full.replace("Journal Jurusan ", "")
            if not prodi_name:
                prodi_name = prodi

            for jurnal in jurnals:
                link = jurnal.select_one('a.gray.button')['href']

                data["no"].append(no)
                data["page"].append(url)          # link halaman
                data["link_keluar"].append(link)  # link detail TA
                no += 1

            # update progress bar
            print_progress(i, prodi_name, j, total_pages)

    df = pd.DataFrame(data)
    df.to_csv("pta_links.csv", index=False)

    end_time = time.time()
    elapsed = int(end_time - start_time)
    jam, sisa = divmod(elapsed, 3600)
    menit, detik = divmod(sisa, 60)

    # summary
    print("\n✅ Seluruh link berhasil dikumpulkan!")
    print(f"📊 Total entri: {len(df)}")
    print(f"⏱️ Waktu eksekusi: {jam} jam {menit} menit {detik} detik")

    return df

In [15]:
# --- Jalankan ---
pta_links()

[1] Ilmu Hukum - Page 3/3 [████████████████████] 100.00%

[2] Teknologi Industri Pertanian - Page 3/3 [████████████████████] 100.00%

[3] Agribisnis - Page 3/3 [████████████████████] 100.00%

[4] Agroteknologi - Page 3/3 [████████████████████] 100.00%

[5] Ilmu Kelautan - Page 3/3 [████████████████████] 100.00%

[6] Ekonomi Pembangunan - Page 3/3 [████████████████████] 100.00%

[7] Manajemen - Page 3/3 [████████████████████] 100.00%

[8] Akuntansi - Page 3/3 [████████████████████] 100.00%

[9] Teknik Industri - Page 3/3 [████████████████████] 100.00%

[10] Teknik Informatika - Page 3/3 [████████████████████] 100.00%

[11] Manajemen Informatika - Page 3/3 [████████████████████] 100.00%

[12] Sosiologi - Page 3/3 [████████████████████] 100.00%

[13] Ilmu Komunikasi - Page 3/3 [████████████████████] 100.00%

[14] Psikologi - Page 3/3 [████████████████████] 100.00%

[15] Sastra Inggris - Page 3/3 [████████████████████] 100.00%

[16] Ekonomi Syariah - Page 3/3 [████████████████████] 100.00%

,no,page,link_keluar
0,1,https://pta.trunojoyo.ac.id/c_search/byprod/1/1,https://pta.trunojoyo.ac.id/welcome/detail/080...
1,2,https://pta.trunojoyo.ac.id/c_search/byprod/1/1,https://pta.trunojoyo.ac.id/welcome/detail/080...
2,3,https://pta.trunojoyo.ac.id/c_search/byprod/1/1,https://pta.trunojoyo.ac.id/welcome/detail/070...
3,4,https://pta.trunojoyo.ac.id/c_search/byprod/1/1,https://pta.trunojoyo.ac.id/welcome/detail/090...
4,5,https://pta.trunojoyo.ac.id/c_search/byprod/1/1,https://pta.trunojoyo.ac.id/welcome/detail/070...
...,...,...,...
476,477,https://pta.trunojoyo.ac.id/c_search/byprod/36/2,https://pta.trunojoyo.ac.id/welcome/detail/160...
477,478,https://pta.trunojoyo.ac.id/c_search/byprod/36/2,https://pta.trunojoyo.ac.id/welcome/detail/160...
478,479,https://pta.trunojoyo.ac.id/c_search/byprod/37/1,https://pta.trunojoyo.ac.id/welcome/detail/170...
479,480,https://pta.trunojoyo.ac.id/c_search/byprod/37/1,https://pta.trunojoyo.ac.id/welcome/detail/170...


## 2. Crawling Berita

### Fungsi

In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [20]:
# Fungsi untuk membersihkan konten dari elemen-elemen yang tidak diinginkan
def clean_content(content_element):
    if not content_element:
        return "Content Not Found"
    return " ".join(content_element.stripped_strings)

# Fungsi untuk mengambil data dari halaman web Detik.com
def get_data(url, kategori, min_articles_per_category):
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return

    soup = BeautifulSoup(response.content, "html.parser")
    articles = soup.find_all("article", class_="list-content__item")

    for article in articles:
        # berhenti kalau artikel per kategori sudah cukup
        if len([k for k in kategori_list if k == kategori]) >= min_articles_per_category:
            return

        try:
            link = article.find("a")["href"]
            article_response = requests.get(link)
            article_response.raise_for_status()
        except (requests.exceptions.RequestException, TypeError) as e:
            print(f"Request for article failed: {e}")
            continue

        article_soup = BeautifulSoup(article_response.content, "html.parser")
        title_element = article_soup.find("h1", class_="detail__title")
        title = title_element.text.strip() if title_element else "Title Not Found"
        content_element = article_soup.find("div", class_="detail__body-text")
        content = clean_content(content_element)

        judul.append(title)
        isi.append(content)
        kategori_list.append(kategori)

        if len(judul) <= 5:  # tampilkan sebagian judul biar tahu sedang jalan
            print("🔎", title)

        time.sleep(1)

# ==============================
# MAIN PROGRAM
# ==============================
start_time = time.time()

# Membuat list url dan kategori yang akan di-crawl
base_urls = [
    "https://health.detik.com/berita-detikhealth/indeks",
    "https://travel.detik.com/travel-news/indeks",
]
categories = [
    "Kesehatan",
    "Pariwisata",
]

# Inisialisasi list untuk menyimpan data
judul = []
isi = []
kategori_list = []

# Batas minimal artikel per kategori
min_articles_per_category = 50

# Melakukan iterasi untuk setiap url dan kategori
for base_url, category in zip(base_urls, categories):
    page = 1
    while len([k for k in kategori_list if k == category]) < min_articles_per_category:
        url = f"{base_url}?page={page}"
        get_data(url, category, min_articles_per_category)
        time.sleep(2)
        page += 1

# Membuat dataframe dengan ID otomatis
df = pd.DataFrame({
    "id_berita": range(1, len(judul) + 1),
    "judul_berita": judul,
    "isi_berita": isi,
    "kategori_berita": kategori_list
})

# Simpan ke CSV
df.to_csv("CrawlBerita_Kesehatan-Pariwisata.csv", index=False, encoding="utf-8-sig")

# Hitung waktu eksekusi
exec_time = time.time() - start_time
minutes, seconds = divmod(int(exec_time), 60)

# ==============================
# TAMPILKAN OUTPUT RINGKASAN
# ==============================
print("✅ Seluruh data berhasil dikumpulkan!")
print(f"📊 Total entri: {len(df)}")
print(f"⏳ Waktu eksekusi: {minutes} menit {seconds} detik")

🔎 Pahit Tapi Sehat,  Ini 5 Manfaat Daun Pepaya Tak Terduga
🔎 Kabar Baik! Racun Lebah Madu Tawarkan Harapan Baru untuk Pengobatan Kanker
🔎 Angka Bunuh Diri di Indonesia Naik, Ini Wilayah yang Catat Kasus Terbanyak
🔎 Geger Dokter di Prancis Racuni Puluhan Pasien Demi Pamer Teknik Resusitasi
🔎 Penampakan Ratusan Benang 'Susuk' Emas di Tubuh Pasien, Masuk Lewat Mana?
✅ Seluruh data berhasil dikumpulkan!
📊 Total entri: 100
⏳ Waktu eksekusi: 4 menit 9 detik


In [21]:
# Tampilkan preview dataframe
display(df.head(10))

,id_berita,judul_berita,isi_berita,kategori_berita
0,1,"Pahit Tapi Sehat, Ini 5 Manfaat Daun Pepaya T...",Daftar Isi Manfaat Daun Pepaya 1. Dapat Menduk...,Kesehatan
1,2,Kabar Baik! Racun Lebah Madu Tawarkan Harapan ...,Jakarta - Sebuah penelitian sedang berlangsung...,Kesehatan
2,3,"Angka Bunuh Diri di Indonesia Naik, Ini Wilaya...",Jakarta - CATATAN: Depresi dan munculnya keing...,Kesehatan
3,4,Geger Dokter di Prancis Racuni Puluhan Pasien ...,Jakarta - Seorang dokter di Prancis dituduh se...,Kesehatan
4,5,Penampakan Ratusan Benang 'Susuk' Emas di Tubu...,"Jakarta - Heboh, dokter di Korea Selatan menem...",Kesehatan
5,6,Temuan Hasil CKG: Depresi-Kecemasan di DKI 10 ...,Jakarta - CATATAN: Depresi dan munculnya keing...,Kesehatan
6,7,Makanan yang Bikin Panjang Umur ala Ahli Gizi ...,Daftar Isi Makanan Apa yang Dikonsumsi? 1. Kac...,Kesehatan
7,8,Ini Tips Jaga Kadar Gula Darah agar Tetap Stab...,Jakarta - Olahraga gym saat ini digemari oleh ...,Kesehatan
8,9,Kemenkes Bicara Alasan Cukai Minuman Berpemani...,Jakarta - Kepala Pusat Kebijakan Ketahanan Kes...,Kesehatan
9,10,"Darurat Kesehatan Mental Gen Z, 720 Ribu Anak ...",Jakarta - CATATAN: Depresi dan munculnya keing...,Kesehatan
